In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/graphics-driver

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-08 15:49:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-01-08 15:49:02 (9.56 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
import pandas as pd
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"),sep="\t",header=True)


In [ ]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
print(f'The row count is {df.count()}')

The row count is 2402458


In [ ]:
df_duplicates_cleaned=df.drop_duplicates()
df_drop_na=df_duplicates_cleaned.dropna()


In [ ]:
df_sql_column=df_drop_na.select(['review_id','customer_id','product_id','product_parent','review_date'])

df_sql_column.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100KOICK3R3CF|   13956383|B000N308RE|     902787115| 2015-01-09|
|R100PWDLZSNOJE|   13214098|B0052UCCI6|      50621170| 2012-11-24|
|R10185C2H88C6O|   30221574|B006TU9EQ6|     520170030| 2014-02-08|
|R102A21SZ34VPM|   39771060|B00CWTZCI2|     489782231| 2014-02-28|
|R102A2XJDY3YPA|   13535367|B0005ZZ53C|     908810009| 2005-09-05|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
mode="append"
jdbc_url="jdbc:postgresql://database-1.co14xqrcwgue.us-west-2.rds.amazonaws.com:5432/bigdata"
config={"user":'postgres',
        "password":'#2020ZuSu',
        "driver":'org.postgresql.Driver'}

In [ ]:
df_sql_column.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   34247947|          1206|
|   17407953|           893|
|   12201275|           737|
|   52545115|           606|
|   53037408|           479|
|   50820654|           458|
|   50199793|           454|
|   37446839|           450|
|    7080939|           445|
|   51812418|           419|
|   50673475|           401|
|   15051518|           397|
|   26221913|           351|
|   39789300|           343|
|   28516018|           316|
|   52989182|           305|
|   44834233|           282|
|   14509190|           278|
|   52833354|           275|
|   52657160|           269|
+-----------+--------------+
only showing top 20 rows



In [ ]:
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088PCJ1C|Bob's Pickle Pops...|
|B005P0U5BO|Envirokidz Organi...|
|B00K2RY8GI|Coffee Variety Sa...|
|B008233EI2|Once Again Organi...|
|B0029K0U90|Mrs. Wages ALL NA...|
|B003H4AMEU|Bhuja Crunchy Sea...|
|B004AJIG76|Krusteaz Belgian ...|
|B002LME6OQ|Prince of Peace O...|
|B00NP7U2U8|Yoki - Cheese Bre...|
|B00ISQD2YO|Nescafe with Coff...|
|B003X35RR2|Bob's Red Mill Gl...|
|B00WVSNJ8Y|Organic Valley, O...|
|B0002UM13C|Sour Cherry Prese...|
|B0042M7PS2|Rodelle Gourmet B...|
|B002DVPTVO|Coffee Go Candy -...|
|B008EL7N2U|Melitta Coffee Gr...|
|B00CCS8Y5Q|Trader Joe's Coco...|
|B006ZOXLF6|Flossugar (1/2 ga...|
|B004RJ0C1W|Camp Chicory & Co...|
|B005JJZ430|KONA BLEND COFFEE...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
df_sql_column.write.jdbc(url=jdbc_url, table='df_sql_column', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)